# Project 3: Collaboration Competition - Report

### Algorithm

The algorithm is DDPG (Deep Deterministic Policy Gradient) applied in a multi agent context. 
The DDPG algorithm is an algorithm used for reinforcement learning that is well suited to estimate continuous actions. It has the following characteristics : 
- model free : there is no model of the environment; the agent uses trial and error learning to produce a policy
- off policy : the policy is not learnt from the current interactions with the environment but from samples of past interactions

<br><br>
Each agent is equipped with its own actor and critic networks (local and target versions). In the multi agent setting, the critic networks are informed about all the state-action pairs from the agents while the actor networks only know the state-action pairs from their respective agent. The following picture displays this architecture (source: <https://papers.nips.cc/paper/7217-multi-agent-actor-critic-for-mixed-cooperative-competitive-environments.pdf>). <br><br>
![Multi Agent Architecture](./marl.png)<br>
 The actor networks learn their optimal policy deterministically. An optimal policy $\pi_{i}$ for an agent $i$ maps a state $s_{i}$ as seen by agent $i$ to the best believed action for this state :
$s_{i} \rightarrow \pi_{i}(s_{i} ; \theta_{\mu_{i}}) $
The critic networks approximate the Q-value function that maps all states and actions to a value :
$s_{1},...,s_{n},a_{1},...,a_{n} \rightarrow Q_{i}(s_{1},...,s_{n},\pi_{1}(s_{1}),...,\pi_{n}(s_{n}); \theta_{Q_i}) $ <br>

In our case, there are two agents ($n = 2$). These agents run episodes of interactions with the environment. These interactions $<s_{1}, s_{2}, a_{1}, a_{2}, r_1, r_2, s'_{1}, s'_{2}>$ are stored in a common replay buffer. Periodically, a batch of interactions is sampled from this buffer and is used to update the networks of the agents this is the learning step and it ocurs in 3 successive substeps : <br><br>

1. Update of the local critic networks: target estimates of the Q values are computed as sums of rewards and discounted estimations of Q values corresponding to next states and estimated next actions, these latter estimations being the results of forward passes on target (actor and critic) networks. A forward pass on the local critic network produces the local estimates of the Q values corresponding to the current states and actions. The differences between local and target estimates are then minimized to update the parameters of the local critic networks. More explicitly : 
    - the actions for the next states are estimated using the target actor networks : $a'_{i} = \pi_{target,i}(s'_{i})$<br>
    - the target Q values for the next states and for the estimated next actions are estimated using the target critic networks: $q'_{i} = Q_{target,i}(s'_{i}, a'_{i})$
    - the target Q values for the current states are computed as the sum of reward and discounted Q values for the next states: $q_{i} = r_{i} + \gamma * q'_{i}$
    - the local Q values for the current states and actions are estimated using the local critic network: $\hat{q_{i}} = Q_{local,i}(s_{i}, a_{i})$
    - the backward passes on the local critic networks compute the respective gradients of the mean squared errors between the estimated Q values : $\nabla_{\theta}\:MSE_{i}(\hat{q_{i}}, q_{i})$
    - these gradients are used to update the weights of the local critic networks (gradient descent).

2. Update of the local actor networks: forward passes on the local (actor and critic) networks produce estimates of Q values corresponding to current states and actions. These estimates are the maximized to update the parameters of the local actor networks. More explicitly :
    - the actions for the current states are predicted using the local actor networks : $a_{pred_{i}} = \pi_{local, i}(s_{i})$ 
    - the Q values corresponding to these predictions are estimated using the local critic networks : $q_{pred_{i}} = Q_{local,i}(s_{i}, a_{pred_{i}})$
    - the backward passes on the local actor networks compute the respective gradients of these Q values : $\nabla_{\theta}\:q_{pred_{i}}$
    - these gradients are used to update the weights of the local actor networks (gradient ascent).

3. Update of the target networks <br> Finally , the parameters of the target networks are progressively aligned on the parameters of the local networks by means of soft updates: $\theta_{target} = \tau * \theta_{local} + (1 - \tau) * \theta_{target}$ 


### Implementation
- Environment : Unity environment provided (Tennis.app)
- Learner : the class equipped with replay buffer, actor and critic networks for each agent. It has the main following methods :
    - act : receives the states and returns the actions with or without noise
    - step : stores an interaction in the replay buffer and periodically samples this buffer to provide a batch of interactions
    - learn : receives this batch of interactions and uses it to update the networks 
- model.py : this file contains the structures of the neural networks (actor & critic). The actor and critic networks are made up of 3 linear layers. The actor maps the states to the actions (policy function $\pi_{i}$) and the critic maps state and action to Q value (size = 1). The numbers of hidden units were chosen based on the state size (24) and action size (2), namely about 10 * (24 + 2). Three linear layers are sufficient for the agents to learn and to reach the desired scores. 
- function train : drives the learner through the main training loop, controls the noise that is injected for exploration and collect the scores for reporting. This function also saves the checkpoints for the local actor networks that cal later be used for testing. 
- noise.py : Ornstein-Uhlenbeck (OU) noise


#### Hyperparameters 
```
n_episodes = 10000     # number of episodes to train
BUFFER_SIZE = int(1e6) # Replay buffer size
BATCH_SIZE = 32        # mini batch size
GAMMA = 0.995          # discount factor
TAU = 9e-3             # for soft update of the target parameters
LR_ACTOR = 5e-5        # Learning rate of actor
LR_CRITIC = 1e-4       # Learning rate of critic
UPDATE_EVERY = 2       # Periodicity : learn & soft updates every 2 interactions

mu = 0.0                 # mean of OU noise (Gaussian distribution)
sigma = 0.5              # variance of OU noise
noise_reduction = 0.9999 # reduction factor applied at each step to the noise
t_stop_noise = 12000     # after this number of steps, no noise injected (with 14 steps in average per episode, noise injected in first 857 episodes)
```

### Results
![](scores.jpg)



### Further Research
- Tuning hyperparameters : in particular the learning rates and the batch size (see training.py)
- Tuning the architectures of the networks : adding an extra layer to the actor networks



In [ ]:
r